<h1 style='text-align:center'>Neighborhood Recommendation Tool</h1>
<h2 style='text-align:center'>Applied Data Science Capstone by IBM/Coursera - The Battle of the Neighborhoods</h2>
<h4 style='text-align:center'>This tool will use foursquare venue data and ratings to recommend a neighborhood to live in based on a user's ranking of neighborhood features.</h4>

<h2>Introduction/Business Problem</h2>

<p>Problem Statement: A user is moving to a new city, and wants to better understand how each neighborhood in that city aligns with the user's particular taste/preference.</p>
<p>Introduction: Using foursquare api venue ratings, each neighborhood in a city is given a score for a set of neighborhood features. The final output will be a map which contains the neighborhood rankings for the user to explore.</p>
<p>For the sake of this exercise, the city will be <b>Washington, DC</b>, and the neighborhood features will be: <b>food, drinks, shops, arts, and outdoors venues.</b></p>
<p>The target audience would be a prospective resident to Washington, DC.</p>

<h2>Data</h2>

Based on the business problem, the following datasets are needed before performing the Content-Based recommendation algorithm to recommend a neighborhood:
1. Washington, DC neighborhoods with latitude, longitude
2. Foursquare venue type rating data associated with Washington, DC neighborhood
3. User input rating of neighborhood features food, drinks, shops, arts, and outdoors venues

<b>Below I will describe in detail the procedure for pulling these datasets.</b>

<h3>Import relevant packages</h3>

In [1]:
##import packages
import pandas as pd
import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Import Complete')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Import Complete


<h3>Dataset 1: Washington, DC neighborhoods with latitude, longitude</h3>
<b>First, I will scrape Washington DC neighborhoods from DC.GOV website using beautifulsoup. Some neighborhoods are not named correctly on the website, these are changed manually.</b>

In [2]:
url = 'https://washington.org/dc-neighborhoods'

html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html.parser")

h5 = soup.findAll('h5')

dc_neighborhoods = []

for i in range(27,51):
    cell={}
    cell['Neighborhood'] = h5[i].text.strip('\n').strip('\n ')
    cell['City'] = 'Washington'
    cell['State'] = 'D.C.'
    dc_neighborhoods.append(cell)

dc_neighborhoods_df = pd.DataFrame(dc_neighborhoods)
dc_neighborhoods_df['Neighborhood']=dc_neighborhoods_df['Neighborhood'].replace({'Explore Dupont Circle':'Dupont Circle',
                                             'Your Guide to the Georgetown Neighborhood':'Georgetown',
                                             'Your Guide to the National Mall':'National Mall',
                                             'Penn Quarter & Chinatown':'Penn Quarter',
                                             'Capitol Riverfront':'Navy Yard',
                                             'Upper Northwest':'Tenleytown',
                                             'U Street':'1200 U St NW',
                                             'Mount Vernon Square':'700 M St NW'})

print(dc_neighborhoods_df.head())
print(dc_neighborhoods_df.shape)

   Neighborhood        City State
0  Adams Morgan  Washington  D.C.
1     Anacostia  Washington  D.C.
2     Brookland  Washington  D.C.
3  Capitol Hill  Washington  D.C.
4     Navy Yard  Washington  D.C.
(24, 3)


<b>Then, I will use geopy to get Lat/Long for each neighborhood</b>

In [3]:
locator = Nominatim(user_agent='dc_explorer')

lat_long = []

for ind in dc_neighborhoods_df.index:
    cell={}
    #print('Neighborhood = {}'.format(dc_neighborhoods_df['Neighborhood'][ind]))
    location = locator.geocode(dc_neighborhoods_df['Neighborhood'][ind]+','+dc_neighborhoods_df['City'][ind]+','+dc_neighborhoods_df['State'][ind])
    #print('Neighborhood = {}, Latitude = {}, Longitude = {}'.format(dc_neighborhoods_df['Neighborhood'][ind],location.latitude, location.longitude))
    cell['Neighborhood'] = dc_neighborhoods_df['Neighborhood'][ind]
    cell['Latitude'] = location.latitude
    cell['Longitude'] = location.longitude
    lat_long.append(cell)
    
lat_long_df = pd.DataFrame(lat_long)

print(lat_long_df.head())
print(lat_long_df.shape)

   Neighborhood   Latitude  Longitude
0  Adams Morgan  38.921500 -77.042199
1     Anacostia  38.862581 -76.984441
2     Brookland  38.932832 -76.984226
3  Capitol Hill  38.889803 -77.009418
4     Navy Yard  38.876307 -77.000478
(24, 3)


<b>Finally, the two dataframes are merged.</b>

In [4]:
dc_df = dc_neighborhoods_df.set_index('Neighborhood').join(lat_long_df.set_index('Neighborhood'))
dc_df.reset_index(inplace=True)
dc_df

,Neighborhood,City,State,Latitude,Longitude
0,Adams Morgan,Washington,D.C.,38.921500,-77.042199
1,Anacostia,Washington,D.C.,38.862581,-76.984441
2,Brookland,Washington,D.C.,38.932832,-76.984226
3,Capitol Hill,Washington,D.C.,38.889803,-77.009418
4,Navy Yard,Washington,D.C.,38.876307,-77.000478
5,Columbia Heights,Washington,D.C.,38.928185,-77.031923
6,Congress Heights,Washington,D.C.,38.842897,-77.000255
7,Downtown,Washington,D.C.,38.900397,-77.028259
8,Dupont Circle,Washington,D.C.,38.912423,-77.041251
9,Foggy Bottom,Washington,D.C.,38.899114,-77.054728


<b>Let's vizualize the 24 nieghborhoods with Folium to see that we have good coverage around the city.</b>

In [5]:
address = 'Washington, D.C.'

location = locator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map 
map_dc = folium.Map(width=800,height=500,location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(dc_df['Latitude'], dc_df['Longitude'], dc_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dc)  
    
map_dc

<b>Now let's see if there is any overlap in the neighborhoods with a radius of 400 meters surrounding the center.</b>

In [6]:
# create map 
map_dc_overlap = folium.Map(width=800,height=500,location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(dc_df['Latitude'], dc_df['Longitude'], dc_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.Circle(
        [lat, lng],
        radius=400, #use circle and set radius to 400m
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dc_overlap)  
    
map_dc_overlap

<b>Looks good! Now let's get the foursqare data</b>
<h3>Dataset 2: Foursquare venue type rating data associated with Washington, DC neighborhood</h3>
<b>First, initiate the credentials (hidden)</b>

In [7]:
# The code was removed by Watson Studio for sharing.

<b>Then, set up the function to get foursquare venues based on category</b>

In [8]:
def getVenues(names, latitudes, longitudes, features, radius=400):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('Pulling venues for: ', name)
        for feat in features:
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}&sortByPopularity=1'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT,
                feat)

            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng,
                feat,
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude',
                  'Feature',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<b>Let's try an example neighborhood to explore how this will work</b>

In [9]:
sample = dc_df.head(1)
print(sample)

neigh_features = ['food', 'drinks', 'shops', 'arts', 'outdoors']

sample_venues = getVenues(names=sample['Neighborhood'], latitudes=sample['Latitude'], longitudes=sample['Longitude'], features=neigh_features)

sample_venues.head()


   Neighborhood        City State  Latitude  Longitude
0  Adams Morgan  Washington  D.C.   38.9215 -77.042199
Pulling venues for:  Adams Morgan


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Feature,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adams Morgan,38.9215,-77.042199,food,Lapis,38.921302,-77.043890,Afghan Restaurant
1,Adams Morgan,38.9215,-77.042199,food,Popeyes Louisiana Kitchen,38.923937,-77.040411,Fried Chicken Joint
2,Adams Morgan,38.9215,-77.042199,food,Amsterdam Falafelshop,38.921162,-77.041959,Falafel Restaurant
3,Adams Morgan,38.9215,-77.042199,food,Mintwood Place,38.922053,-77.043611,New American Restaurant
4,Adams Morgan,38.9215,-77.042199,food,So's Your Mom,38.921671,-77.043753,Bagel Shop


<b>Let's visualize these venues on the map. Food will be Blue, Drinks Green, Shops Purple, Arts Red, and Outdoors Orange</b>

In [10]:
feature_to_color= {'food':'blue','drinks':'purple','shops':'green','arts':'red','outdoors':'orange'}


latitude = sample['Latitude'][0]
longitude = sample['Longitude'][0]

# create map 
map_admo = folium.Map(width=800,height=500,location=[latitude, longitude], zoom_start=16)

# add markers to map
for lat, lng, venue, feat in zip(sample_venues['Venue Latitude'], sample_venues['Venue Longitude'], sample_venues['Venue'], sample_venues['Feature']):
    label = '{}, {}'.format(venue, feat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=feature_to_color[feat],
        fill=True,
        fill_color=feature_to_color[feat],
        fill_opacity=0.3,
        parse_html=False).add_to(map_admo)  
    
map_admo

<b>Now let's see the breakup by feature for this neighborood.</b>

In [11]:
#sample_venues[sample_venues['Feature']=='arts']
sample_venues.groupby('Feature').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Feature,,,,,,,
arts,4,4,4,4,4,4,4
drinks,29,29,29,29,29,29,29
food,40,40,40,40,40,40,40
outdoors,19,19,19,19,19,19,19
shops,74,74,74,74,74,74,74


<b>Once we do this for all the neighborhoods and normalize, we can do the comparison and assign a rating for each feature for each neighborhood.</b>
<p>
<b>Now, let's repeat with all neighborhoods</b>

In [12]:
full_venues = getVenues(names=dc_df['Neighborhood'], latitudes=dc_df['Latitude'], longitudes=dc_df['Longitude'], features=neigh_features)
print('Venues Pulled')

Pulling venues for:  Adams Morgan
Pulling venues for:  Anacostia
Pulling venues for:  Brookland
Pulling venues for:  Capitol Hill
Pulling venues for:  Navy Yard
Pulling venues for:  Columbia Heights
Pulling venues for:  Congress Heights
Pulling venues for:  Downtown
Pulling venues for:  Dupont Circle
Pulling venues for:  Foggy Bottom
Pulling venues for:  Georgetown
Pulling venues for:  H Street NE
Pulling venues for:  Ivy City
Pulling venues for:  Logan Circle
Pulling venues for:  700 M St NW
Pulling venues for:  National Mall
Pulling venues for:  NoMa
Pulling venues for:  Penn Quarter
Pulling venues for:  Petworth
Pulling venues for:  Shaw
Pulling venues for:  Southwest & The Wharf
Pulling venues for:  1200 U St NW
Pulling venues for:  Tenleytown
Pulling venues for:  Woodley Park
Venues Pulled


In [13]:
print(full_venues.groupby(['Neighborhood','Feature']).count().to_string())

                                Neighborhood Latitude  Neighborhood Longitude  Venue  Venue Latitude  Venue Longitude  Venue Category
Neighborhood          Feature                                                                                                        
1200 U St NW          arts                         16                      16     16              16               16              16
                      drinks                       57                      57     57              57               57              57
                      food                         60                      60     60              60               60              60
                      outdoors                     26                      26     26              26               26              26
                      shops                        75                      75     75              75               75              75
700 M St NW           arts                          7         

<b>Great, now we will get the user input rating for these features.</b>
<h3>Dataset 3: User input rating of neighborhood features food, drinks, shops, arts, and outdoors venues</h3>

In [74]:
table_contents=[]
cell={}

for x in neigh_features:
    val = input('On a scale of 1 (least important) to 5 (most important), how important are '+x+' venues to you: ')
    cell[x] = int(val)
    
table_contents.append(cell)
    
user_df=pd.DataFrame(table_contents)
user_df

On a scale of 1 (least important) to 5 (most important), how important are food venues to you: 1
On a scale of 1 (least important) to 5 (most important), how important are drinks venues to you: 1
On a scale of 1 (least important) to 5 (most important), how important are shops venues to you: 5
On a scale of 1 (least important) to 5 (most important), how important are arts venues to you: 1
On a scale of 1 (least important) to 5 (most important), how important are outdoors venues to you: 1


,food,drinks,shops,arts,outdoors
0,1,1,5,1,1


<b>This concludes the Week 1 work for the Data Science Capstone, next steps are to describe and apply a methodology to the above datasets to arrive at a recommendation for the user, present the results, and conclude the report.</b>

<h2>Methodology</h2>
<p>
In this project we will direct our efforts on identifying neighborhoods of Washington DC that have venue characteristics which align to the user's preference. 

In first step we have collected the required data: neighborhood location and type (category) every venue within 400m of the neighborhood center.

The next step will be to normalize this neighborhood venue data to draw feature insights for each Washington DC neighborhood. For the sake of this exersise, the <b>number</b> of venues in each type will be used as the indicator of that type's prevalence in the area (versus something like average rating/proximity to the center).

In third and final step we will bring in the user's preferences to see which neighborhood is the best fit based on the preferences, and display that neighborhood with the venues for the user to explore.
</p>

<h2>Analysis</h2>

<b>First, take the percentage of venues in each neighborhood</b>

In [149]:
neighborhood_venue = []

for n in full_venues['Neighborhood'].unique():
    cell={}
    #print(n, ': ', len(full_venues[full_venues['Neighborhood']==n]), ' total venues')
    cell['Neighborhood'] = n
    for f in neigh_features:
        #print('\t # ', f, ' venues: ', len(full_venues[(full_venues['Neighborhood']==n)&(full_venues['Feature']==f)]))
        cell[f] = len(full_venues[(full_venues['Neighborhood']==n)&(full_venues['Feature']==f)]) / len(full_venues[full_venues['Neighborhood']==n])
    neighborhood_venue.append(cell)    

    
neighborhood_venue_df = pd.DataFrame(neighborhood_venue)

print(neighborhood_venue_df.to_string())

             Neighborhood      food    drinks     shops      arts  outdoors
0            Adams Morgan  0.240964  0.174699  0.445783  0.024096  0.114458
1               Anacostia  0.000000  0.000000  0.833333  0.166667  0.000000
2               Brookland  0.000000  0.000000  0.600000  0.000000  0.400000
3            Capitol Hill  0.296296  0.000000  0.222222  0.111111  0.370370
4               Navy Yard  0.275000  0.075000  0.312500  0.050000  0.287500
5        Columbia Heights  0.273684  0.021053  0.505263  0.052632  0.147368
6        Congress Heights  0.210526  0.000000  0.578947  0.105263  0.105263
7                Downtown  0.126374  0.076923  0.549451  0.060440  0.186813
8           Dupont Circle  0.208589  0.116564  0.503067  0.061350  0.110429
9            Foggy Bottom  0.160000  0.080000  0.260000  0.280000  0.220000
10             Georgetown  0.196970  0.101010  0.505051  0.060606  0.136364
11            H Street NE  0.245098  0.068627  0.450980  0.068627  0.166667
12          

<b>Then, compare these with the other neighborhoods to see how each neighborhood makeup compares across the city</b>

In [150]:
neighborhood_venue = []

i=0
for row in neighborhood_venue_df['Neighborhood']:
    cell={}
    cell['Neighborhood'] = row
    for f in neigh_features:
        cell[f+'_weighted']= neighborhood_venue_df[(neighborhood_venue_df['Neighborhood']==row)][f][i] / neighborhood_venue_df[f].sum()
    i=i+1
    neighborhood_venue.append(cell)

neighborhood_venue_df = pd.DataFrame(neighborhood_venue) 
    
neighborhood_venue = []

i=0
for row in neighborhood_venue_df['Neighborhood']:
    cell={}
    cell['Neighborhood'] = row
    for f in neigh_features:
        cell[f+'_normalized']= neighborhood_venue_df[(neighborhood_venue_df['Neighborhood']==row)][f+'_weighted'][i] / neighborhood_venue_df[f+'_weighted'].max()
    i=i+1
    neighborhood_venue.append(cell)
    
neighborhood_venue_df = pd.DataFrame(neighborhood_venue)

print(neighborhood_venue_df.to_string())  

             Neighborhood  food_normalized  drinks_normalized  shops_normalized  arts_normalized  outdoors_normalized
0            Adams Morgan         0.593804           0.717185          0.534940         0.086059             0.190763
1               Anacostia         0.000000           0.000000          1.000000         0.595238             0.000000
2               Brookland         0.000000           0.000000          0.720000         0.000000             0.666667
3            Capitol Hill         0.730159           0.000000          0.266667         0.396825             0.617284
4               Navy Yard         0.677679           0.307895          0.375000         0.178571             0.479167
5        Columbia Heights         0.674436           0.086427          0.606316         0.187970             0.245614
6        Congress Heights         0.518797           0.000000          0.694737         0.375940             0.175439
7                Downtown         0.311421           0.3

<b>Now, take the user's rating input as the weights against these scores to recommend a particular neighborhood</b>

In [95]:
recommendation_table = []

i=0
for row in neighborhood_venue_df['Neighborhood']:
    cell={}
    cell['Neighborhood'] = row
    x = 0
    y = 0
    for f in neigh_features:
        x = x + ((neighborhood_venue_df[(neighborhood_venue_df['Neighborhood']==row)][f+'_normalized'][i])*(user_df[f][0]-1))
        y = y + (user_df[f][0]-1)
    i=i+1
    cell['Score'] = x / y
    recommendation_table.append(cell)


recommendationTable_df = pd.DataFrame(recommendation_table).sort_values('Score', ascending = False)
recommendationTable_df = recommendationTable_df.reset_index().drop('index',axis=1)
recommendationTable_df

,Neighborhood,Score
0,Anacostia,1.000000
1,Brookland,0.720000
2,Congress Heights,0.694737
3,Downtown,0.659341
4,Columbia Heights,0.606316
5,Georgetown,0.606061
6,Dupont Circle,0.603681
7,Petworth,0.600000
8,H Street NE,0.541176
9,Adams Morgan,0.534940


<b>Let's generalize this into a function</b>

In [119]:
def userInputRatings():
    neigh_features = ['food', 'drinks', 'shops', 'arts', 'outdoors']
    
    table_contents=[]
    cell={}

    for x in neigh_features:
        val = input('On a scale of 1 (least important) to 5 (most important), how important are '+x+' venues to you: ')
        cell[x] = int(val)

    table_contents.append(cell)

    input_df=pd.DataFrame(table_contents)
    return input_df

def neighborhoodRecommendation(ratings):
    recommendation_table = []

    i=0
    for row in neighborhood_venue_df['Neighborhood']:
        cell={}
        cell['Neighborhood'] = row
        x = 0
        y = 0
        for f in neigh_features:
            x = x + ((neighborhood_venue_df[(neighborhood_venue_df['Neighborhood']==row)][f+'_normalized'][i])*(ratings[f][0]-1))
            y = y + (ratings[f][0]-1)
        i=i+1
        cell['Score'] = x / y
        recommendation_table.append(cell)


    recommendationTable_df = pd.DataFrame(recommendation_table).sort_values('Score', ascending = False)
    recommendationTable_df = recommendationTable_df.reset_index().drop('index',axis=1)
    return recommendationTable_df.head(5)

In [122]:
ratings_df = userInputRatings()
recommendation = neighborhoodRecommendation(ratings_df)
recommendation

On a scale of 1 (least important) to 5 (most important), how important are food venues to you: 4
On a scale of 1 (least important) to 5 (most important), how important are drinks venues to you: 5
On a scale of 1 (least important) to 5 (most important), how important are shops venues to you: 4
On a scale of 1 (least important) to 5 (most important), how important are arts venues to you: 2
On a scale of 1 (least important) to 5 (most important), how important are outdoors venues to you: 3


,Neighborhood,Score
0,1200 U St NW,0.589540
1,Adams Morgan,0.517119
2,Shaw,0.512333
3,Ivy City,0.509959
4,700 M St NW,0.480602


<b>Define a function to map out recommended neighborhood</b>

In [139]:
def mapRecommendation(neighborhood):
    feature_to_color= {'food':'blue','drinks':'purple','shops':'green','arts':'red','outdoors':'orange'}
    rec_df = full_venues[full_venues['Neighborhood']==neighborhood]
    latitude = rec_df['Neighborhood Latitude'].mean()
    longitude = rec_df['Neighborhood Longitude'].mean()

    # create map 
    map_rec = folium.Map(width=800,height=500,location=[latitude, longitude], zoom_start=16)

    # add markers to map
    for lat, lng, venue, feat in zip(rec_df['Venue Latitude'], rec_df['Venue Longitude'], rec_df['Venue'], rec_df['Feature']):
        label = '{}, {}'.format(venue, feat)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=feature_to_color[feat],
            fill=True,
            fill_color=feature_to_color[feat],
            fill_opacity=0.3,
            parse_html=False).add_to(map_rec)  

    return map_rec

<h2>Results and Discussion</h2>

A person who is moving into Washington, DC who prioritizes drinks venues the most, food and shopping second, outdoors third and arts last should ideally move into the <b>U Street</b> area, followed closely by the <b>Adams Morgan</b> and <b>Shaw</b> areas. This can be seen below:

In [152]:
ratings_df = userInputRatings()
recommendation = neighborhoodRecommendation(ratings_df)
print(recommendation)
map_final = mapRecommendation((recommendation['Neighborhood'][0]))
map_final

On a scale of 1 (least important) to 5 (most important), how important are food venues to you: 4
On a scale of 1 (least important) to 5 (most important), how important are drinks venues to you: 5
On a scale of 1 (least important) to 5 (most important), how important are shops venues to you: 4
On a scale of 1 (least important) to 5 (most important), how important are arts venues to you: 2
On a scale of 1 (least important) to 5 (most important), how important are outdoors venues to you: 3
   Neighborhood     Score
0  1200 U St NW  0.589540
1  Adams Morgan  0.517119
2          Shaw  0.512333
3      Ivy City  0.509959
4   700 M St NW  0.480602


#### The above map can be explored by the user to see what is in the recommended area based on foursquare venue data.

<b>Observations</b>

While performing the exploratory analysis on the DC neighborhoods, I had the following observations:
1. Anacostia, Brookland, and National Mall areas all had 10 or less venues when pulled from foursquare, which could skew the algorithm.

In [145]:
full_venues.groupby(['Neighborhood']).count()[full_venues.groupby(['Neighborhood']).count()['Neighborhood Latitude']<=10]

,Neighborhood Latitude,Neighborhood Longitude,Feature,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Anacostia,6,6,6,6,6,6,6
Brookland,5,5,5,5,5,5,5
National Mall,10,10,10,10,10,10,10


2. Minor issues with the foursquare data. Some foursquare categorizations of the venues could be off, as a restaurant can also be a place to get drinks, etc. Also, some venues come up multiple times due to multiple entries in foursquare.

3. The neighborhood vicinities were taken based off of radius from the center. In reality, neighborhoods are not perfect circles, but rather they are imperfect shapes. Ideally, venues should be considered which fall out of the radius but are realistically in the neighborhood.

<h2>Conclusion</h2>

Despite the difficulties presented in the Discussion section, the algorithm/model and functions serve the purpose proposed in the business problem. As a resident of Washington DC for the past 2 years, I feel the ratings which my algorithm came up with are fairly accurate with what I know of the neighborhoods in the area. It was particularly interesting to me that the algorithm recommended me to live in the U Street Area instead of my actual neighborhood, Adams Morgan.

I would advise any prospective resident to the area to play around with the user rankings and see the different neighborhoods it recommends; it would be a great first step in finding 2-3 neighborhoods to continue researching.